This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [12]:
# Store the API key as a string - according to PEP8, constants are always named
# in all upper case

with open('quandl_api_key.txt', 'r') as f:
    API_KEY = f.readline()

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [25]:
# First, import the relevant modules
import requests
import json
import pandas as pd
import collections

In [41]:
# Now, call the Quandl API and pull out a small sample of the data (only one day)
# to get a glimpse into the JSON structure that will be returned

# Construct the url
base_url = 'https://www.quandl.com/api/v3/datasets/{database_code}/{dataset_code}/'\
           'data.json?api_key={api_key}&start_date={start_date}&end_date={end_date}'
url = base_url.format(database_code='FSE', dataset_code='AFX_X', api_key=API_KEY,
                      start_date='2020-04-29', end_date='2020-04-29')

# Retrieve the data
res = requests.get(url)
json_obj = res.json()

# Print out in a pretty format
print(json.dumps(json_obj, indent=2))

{
  "dataset_data": {
    "limit": null,
    "transform": null,
    "column_index": null,
    "column_names": [
      "Date",
      "Open",
      "High",
      "Low",
      "Close",
      "Change",
      "Traded Volume",
      "Turnover",
      "Last Price of the Day",
      "Daily Traded Units",
      "Daily Turnover"
    ],
    "start_date": "2020-04-29",
    "end_date": "2020-04-29",
    "frequency": "daily",
    "data": [
      [
        "2020-04-29",
        90.0,
        91.0,
        87.85,
        90.95,
        null,
        143888.0,
        12901979.9,
        null,
        null,
        null
      ]
    ],
    "collapse": null,
    "order": null
  }
}


In [39]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

Data is contained under 'dataset_date' key and consists of the following fields.

**Field information**
Source: https://docs.quandl.com/docs/parameters-2

- limit: int. meaning extracted first n rows.
- transform: str. show if elementary calculation was performed.
- column_indx: int. if columns to extract specified
- column_names: list of str, column headers
- start_date: str. Date the first data retrieved is on (inclusive).
- end_date: str. Date the last data retrieved is on (inclusive).
- frequency: str. Frequency to collapse data. Can be daily or longer.
- data: list of any types. values.
- collapse: str. data frequency if data is collapsed, none means as in the original.
- order: str. ascending or descending.

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [94]:
# Step 1: Retrieve data
url = base_url.format(database_code='FSE', dataset_code='AFX_X', api_key=API_KEY,
                      start_date='2017-01-01', end_date='2017-12-31')
res = requests.get(url)  # Retrieve data
json_data = res.json()   # Convert retrieved text into JSON (=dict)
json_data = json_data['dataset_data']  # unnest the first key

# Step 2: Create a DataFrame with the retrived JSON data
df = pd.DataFrame(json_data['data'], columns=json_data['column_names'])

In [96]:
# Check the head of df
df.head()

,Date,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
0,2017-12-29,51.76,51.94,51.45,51.76,NaN,34640.0,1792304.0,None,None,None
1,2017-12-28,51.65,51.82,51.43,51.60,NaN,40660.0,2099024.0,None,None,None
2,2017-12-27,51.45,51.89,50.76,51.82,NaN,57452.0,2957018.0,None,None,None
3,2017-12-22,51.05,51.50,50.92,51.32,NaN,71165.0,3641949.0,None,None,None
4,2017-12-21,51.16,51.52,50.90,51.40,NaN,120649.0,6179433.0,None,None,None


In [98]:
# Check for Null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   255 non-null    object 
 1   Open                   252 non-null    float64
 2   High                   255 non-null    float64
 3   Low                    255 non-null    float64
 4   Close                  255 non-null    float64
 5   Change                 1 non-null      float64
 6   Traded Volume          255 non-null    float64
 7   Turnover               255 non-null    float64
 8   Last Price of the Day  0 non-null      object 
 9   Daily Traded Units     0 non-null      object 
 10  Daily Turnover         0 non-null      object 
dtypes: float64(7), object(4)
memory usage: 22.0+ KB


In [99]:
# Find where Open variable is NaN (not going to remove them though)
df[df.Open.isnull()]

,Date,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
169,2017-05-01,NaN,42.245,41.655,41.72,-0.44,86348.0,3606589.0,None,None,None
179,2017-04-17,NaN,42.480,41.985,42.20,NaN,88416.0,3734717.0,None,None,None
180,2017-04-14,NaN,42.480,41.985,42.20,NaN,88416.0,3734717.0,None,None,None


In [112]:
# Step 3: Find the highest and lowest opening price
highest = df.Open.max()
lowest = df.Open.min()
print(f'Step 3: Highest and lowest opening price in 2017\n'\
      f'Highest: ${highest:.2f}\nLowest : ${lowest:.2f}')

Step 3: Highest and lowest opening price in 2017
Highest: $53.11
Lowest : $34.00


In [111]:
# Step 4: The largest change in any one day.
df['Diff'] = df['High'] - df['Low']

idx_lc = df['Diff'].idxmax()  # index of the day with the largest diff
date_lc = df.iloc[idx].Date
diff_lc = df.iloc[idx].Diff

print(f'Step 4: Lagest change in any one day\n{date_lc:}: ${diff_lc:,.2f}')

Step 4: Lagest change in any one day
2017-05-11: $2.81


In [105]:
# Step 5: What was the largest change between any two days (based on Closing Price)?
df['Delta'] = df.Close.diff(periods=-1)  # Note df.Date is in the descending order
df['Delta_abs'] = df.Delta.abs()  # To find the largest diff, we'll check abs. diff

idx_md = df.Delta_abs.idxmax()  # index of the 'to' date with the largest diff
date_md_from = df.Date.iloc[idx_md+1]
date_md_to = df.Date.iloc[idx_md]
delta_md = df.Delta.iloc[idx_md]

print(f'Step 5: Largest change between 2 days\n'\
      f'{date_md_from:}~{date_md_to}: ${delta_md:.2f}')

Step 5: Largest change between 2 days
2017-08-08~2017-08-09: $-2.56


In [115]:
# Step 6. What was the average daily trading volume during this year?
print('Step 6. The average daily trading volume: {:,.0f}'\
      .format(df['Traded Volume'].mean()))

Step 6. The average daily trading volume: 89,124


In [118]:
# Step 7. What was the median trading volume during this year. 
print('Step 7: Median trading volume during this year: {:,.0f}'\
      .format(df['Traded Volume'].median()))

Step 7: Median trading volume during this year: 76,286
